In [1]:
#Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyairports.airports import Airports
from datetime import datetime

# Importing Dataset
data=pd.read_csv('Data/Flight_Data_New.csv',usecols=['searchDate', 'flightDate', 'startingAirport',
       'starting_airport_name', 'destinationAirport',
       'destination_airport_name', 'fareBasisCode', 
       'travelDuration', 'isBasicEconomy', 'isRefundable', 'isNonStop',
       'seatsRemaining', 'totalTravelDistance', 'segmentsDepartureTimeRaw',
       'segmentsArrivalTimeRaw',
       'segmentsArrivalAirportCode', 'segmentsDepartureAirportCode',
       'segmentsAirlineName', 'segmentsAirlineCode',
       'segmentsEquipmentDescription', 'segmentsDurationInSeconds',
       'segmentsDistance', 'segmentsCabinCode','totalFare'],nrows=1000)
data.head()

,searchDate,flightDate,startingAirport,starting_airport_name,destinationAirport,destination_airport_name,fareBasisCode,travelDuration,isBasicEconomy,isRefundable,...,segmentsDepartureTimeRaw,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,2022-05-29,2022-06-18,LAX,Los Angeles Intl,ATL,Hartsfield Jackson Atlanta Intl,Y0N,PT4H19M,False,False,...,2022-06-18T15:30:00.000-07:00,2022-06-18T22:49:00.000-04:00,ATL,LAX,Delta,DL,Airbus A321,15540,1943,coach
1,2022-05-03,2022-06-24,CLT,Charlotte Douglas Intl,LAX,Los Angeles Intl,MA0OA0MQ,PT8H10M,False,False,...,2022-06-24T08:30:00.000-04:00||2022-06-24T12:0...,2022-06-24T09:44:00.000-04:00||2022-06-24T13:4...,ATL||LAX,CLT||ATL,Delta||Delta,DL||DL,Boeing 717||Airbus A321,4440||16680,228||1943,coach||coach
2,2022-05-02,2022-06-30,LGA,La Guardia,ATL,Hartsfield Jackson Atlanta Intl,LAVQA0ML,PT11H54M,False,False,...,2022-06-30T19:59:00.000-04:00||2022-07-01T06:0...,2022-06-30T23:16:00.000-04:00||2022-07-01T07:5...,MIA||ATL,LGA||MIA,Delta||Delta,DL||DL,Boeing 737-800||Boeing 737-900,11820||6780,1104||596,coach||coach
3,2022-06-10,2022-08-04,LAX,Los Angeles Intl,SFO,San Francisco Intl,MH7OASMN,PT1H20M,False,False,...,2022-08-04T19:00:00.000-07:00,2022-08-04T20:20:00.000-07:00,SFO,LAX,Alaska Airlines,AS,Embraer 175,4800,339,coach
4,2022-06-24,2022-07-20,LGA,La Guardia,DFW,Dallas Fort Worth Intl,LAVQA0BQ,PT6H41M,True,False,...,2022-07-20T10:35:00.000-04:00||2022-07-20T14:5...,2022-07-20T13:14:00.000-04:00||2022-07-20T16:1...,ATL||DFW,LGA||ATL,Delta||Delta,DL||DL,Airbus A320||Airbus A321,9540||8460,762||725,coach||coach


In [2]:
# Extracting Number of days between flight Date and search date
num_days=[]
for dt in range(data.shape[0]):
    num_day=datetime.strptime(data['flightDate'][dt], '%Y-%m-%d').date()-datetime.strptime(data['searchDate'][dt], '%Y-%m-%d').date()
    num_day=num_day.days
    num_days.append(num_day)
data['NumDays']=num_days

column_index = data.columns.get_loc('flightDate') + 1
data.insert(column_index, 'NumDays', data.pop('NumDays'))

data.head()

,searchDate,flightDate,NumDays,startingAirport,starting_airport_name,destinationAirport,destination_airport_name,fareBasisCode,travelDuration,isBasicEconomy,...,segmentsDepartureTimeRaw,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,2022-05-29,2022-06-18,20,LAX,Los Angeles Intl,ATL,Hartsfield Jackson Atlanta Intl,Y0N,PT4H19M,False,...,2022-06-18T15:30:00.000-07:00,2022-06-18T22:49:00.000-04:00,ATL,LAX,Delta,DL,Airbus A321,15540,1943,coach
1,2022-05-03,2022-06-24,52,CLT,Charlotte Douglas Intl,LAX,Los Angeles Intl,MA0OA0MQ,PT8H10M,False,...,2022-06-24T08:30:00.000-04:00||2022-06-24T12:0...,2022-06-24T09:44:00.000-04:00||2022-06-24T13:4...,ATL||LAX,CLT||ATL,Delta||Delta,DL||DL,Boeing 717||Airbus A321,4440||16680,228||1943,coach||coach
2,2022-05-02,2022-06-30,59,LGA,La Guardia,ATL,Hartsfield Jackson Atlanta Intl,LAVQA0ML,PT11H54M,False,...,2022-06-30T19:59:00.000-04:00||2022-07-01T06:0...,2022-06-30T23:16:00.000-04:00||2022-07-01T07:5...,MIA||ATL,LGA||MIA,Delta||Delta,DL||DL,Boeing 737-800||Boeing 737-900,11820||6780,1104||596,coach||coach
3,2022-06-10,2022-08-04,55,LAX,Los Angeles Intl,SFO,San Francisco Intl,MH7OASMN,PT1H20M,False,...,2022-08-04T19:00:00.000-07:00,2022-08-04T20:20:00.000-07:00,SFO,LAX,Alaska Airlines,AS,Embraer 175,4800,339,coach
4,2022-06-24,2022-07-20,26,LGA,La Guardia,DFW,Dallas Fort Worth Intl,LAVQA0BQ,PT6H41M,True,...,2022-07-20T10:35:00.000-04:00||2022-07-20T14:5...,2022-07-20T13:14:00.000-04:00||2022-07-20T16:1...,ATL||DFW,LGA||ATL,Delta||Delta,DL||DL,Airbus A320||Airbus A321,9540||8460,762||725,coach||coach


In [3]:
# Flight Day (M,T,W,etc)

data['flightDate'] = pd.to_datetime(data['flightDate'])
data['Day_of_Flight'] = data['flightDate'].dt.day_name()  


column_index = data.columns.get_loc('flightDate') + 1
data.insert(column_index, 'Day_of_Flight', data.pop('Day_of_Flight'))

In [25]:
#Number of Hours
def calculate_total_minutes(duration):
    hour=0
    min=0
    try: 
        hour=int(duration.split('H')[0][-1])
        try:
            min= int(duration.split('H')[1][:-1])
        except:
            min=0
    except:
        try:
            min= int(duration.split('H')[1][:-1])
        except:
            min=0
    return (hour*60)+min

data['Flight_time_in_minutes'] = data['travelDuration'].apply(calculate_total_minutes)

column_index = data.columns.get_loc('travelDuration') + 1
data.insert(column_index, 'Flight_time_in_minutes', data.pop('Flight_time_in_minutes'))

data.drop(['travelDuration'],axis=1,inplace=True) 

In [26]:
data.head()

,searchDate,flightDate,Day_of_Flight,NumDays,startingAirport,starting_airport_name,destinationAirport,destination_airport_name,fareBasisCode,Flight_time_in_minutes,...,segmentsDepartureTimeRaw,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,2022-05-29,2022-06-18,Saturday,20,LAX,Los Angeles Intl,ATL,Hartsfield Jackson Atlanta Intl,Y0N,259,...,2022-06-18T15:30:00.000-07:00,2022-06-18T22:49:00.000-04:00,ATL,LAX,Delta,DL,Airbus A321,15540,1943,coach
1,2022-05-03,2022-06-24,Friday,52,CLT,Charlotte Douglas Intl,LAX,Los Angeles Intl,MA0OA0MQ,490,...,2022-06-24T08:30:00.000-04:00||2022-06-24T12:0...,2022-06-24T09:44:00.000-04:00||2022-06-24T13:4...,ATL||LAX,CLT||ATL,Delta||Delta,DL||DL,Boeing 717||Airbus A321,4440||16680,228||1943,coach||coach
2,2022-05-02,2022-06-30,Thursday,59,LGA,La Guardia,ATL,Hartsfield Jackson Atlanta Intl,LAVQA0ML,114,...,2022-06-30T19:59:00.000-04:00||2022-07-01T06:0...,2022-06-30T23:16:00.000-04:00||2022-07-01T07:5...,MIA||ATL,LGA||MIA,Delta||Delta,DL||DL,Boeing 737-800||Boeing 737-900,11820||6780,1104||596,coach||coach
3,2022-06-10,2022-08-04,Thursday,55,LAX,Los Angeles Intl,SFO,San Francisco Intl,MH7OASMN,80,...,2022-08-04T19:00:00.000-07:00,2022-08-04T20:20:00.000-07:00,SFO,LAX,Alaska Airlines,AS,Embraer 175,4800,339,coach
4,2022-06-24,2022-07-20,Wednesday,26,LGA,La Guardia,DFW,Dallas Fort Worth Intl,LAVQA0BQ,401,...,2022-07-20T10:35:00.000-04:00||2022-07-20T14:5...,2022-07-20T13:14:00.000-04:00||2022-07-20T16:1...,ATL||DFW,LGA||ATL,Delta||Delta,DL||DL,Airbus A320||Airbus A321,9540||8460,762||725,coach||coach
